In [2]:
# !/usr/bin/env python
# -*- coding: utf-8 -*-
#
# Author: David Hokken
# Email:  david.hokken@gmail.com
# Date:   28/9/2020
#


###############################
########### MODULES ########### 
###############################


import itertools
import math
import matplotlib.pyplot as plt
import numpy as np


###############################
########## FUNCTIONS ##########
###############################


######################
#### Basic things ####
######################

R.<x> = PolynomialRing(QQ)
S.<y> = PolynomialRing(QQ)

def F(n):
    '''Returns a list with all Littlewood polynomials of degree n'''
    L = (itertools.product((1,-1),repeat=n))
    L = [x^n+R(v) for v in L]
    return L

def get_ir(lst):
    '''Returns the irreducibles from a list of polynomials'''
    return [f for f in lst if f.is_irreducible()]

def get_sq(lst):
    '''Returns the polynomials with square discriminant from a list of polynomials'''
    return [f for f in lst if f.discriminant().is_square()]

def get_sqir(lst):
    '''Returns the irreducibles with square discriminant from a list of polynomials
    NOTE: get_ir(get_sq(...)) is MUCH faster than get_sq(get_ir(...))'''
    return get_ir(get_sq(lst))

def Ir(n):
    '''Returns a list with all irreducible Littlewood polynomials of degree n'''
    return get_ir(F(n))

def Sq(n):
    '''Returns a list with all irreducible Littlewood polynomials of degree n with square discriminant'''
    return get_sqir(F(n))

def Sqnotir(n):
    '''Returns a list with all Littlewood polynomials of degree n with square discriminant'''
    return get_sqs(F(n))


############################
#### (skew-)reciprocals ####
############################

def PaL(n):
    '''Returns a list of all reciprocal Littlewood polynomials of degree 2n'''
    L = list(itertools.product((1,-1),repeat=n-1))
    PaL = []
    for i in range(len(L)):
        L[i] = list(L[i])
        L[i].extend([1,1])
        L[i] = tuple(L[i])
        PaL.append(list(reversed(L[i]))[:-1])
        PaL[i].extend(list(L[i]))
    PaL = [R(v) for v in PaL]
    return PaL 

def Palnotsq(n):
    '''Returns a list of all irreducible reciprocal Littlewood polynomials of degree 2n'''
    return get_ir(PaL(n))

def Pal(n):
    '''Returns a list of all irreducible reciprocal Littlewood polynomials of degree 2n with square discriminant'''
    return get_sqir(PaL(n))

def ISP(n):
    '''Returns a list of all irreducible skewreciprocal Littlewood polynomials of degree 2n'''
    L = list(itertools.product((1,-1),repeat=n-1))
    ISPlist = []
    for i in range(len(L)):
        lst = list(L[i])
        M = lst[::-1]
        M.extend([1,1])
        N = [-1*x for x in M[1::2]]
        M = M[::2]
        M = [x for y in zip(M, N) for x in y]
        if n % 4 == 0:
            M.append(1)
        lst.extend(M[1:])
        lst[0:0]=[1,1]
        lst = tuple(reversed(lst))
        ISPlist.append(lst)
    ISPlist = get_sqir([R(v) for v in ISPlist])
    return ISPlist


#######################
#### Miscellaneous ####
#######################

def bintopol(number):
    '''Given a string of 0s and 1s, converts the 0s to -1s and converts the resulting string of 1s and -1s to
    a Littlewood polynomial of degree the length of the string minus 1 with i-th coefficients corresponding to
    the i-th element in the string'''
    digits = [int(x) for x in str(number)]
    for i in range(len(digits)):
        if digits[i] == 0:
            digits[i] = -1
    return R(digits)

def bintodisc(number):
    '''Does like bintopol, but in addition, returns whether the resulting polynomial is irreducible and has
    square discriminant'''
    return (bintopol(number), bintopol(number).is_irreducible(), bintopol(number).discriminant().is_square())

def is_palindrome(f):
    '''Checks if a given polynomial is reciprocal'''
    return f.list() == list(reversed(f.list()))

def realandcplx(lst):
    '''Given a list of polynomials, returns for each polynomial in the list its index, number of real roots, number
    of unimodular roots, whether it is reciprocal or not, the polynomial itself, and a list of tuples of the roots
    (in the real case) or the absolute value of the root (in the complex case) with specification of them being 
    either real or complex'''
    for i in range(len(lst)):
        lst2 = lst[i].roots(ring=CDF)
        numberofreals = 0
        numberofunits = 0
        for n in range(len(lst2)):
            if lst2[n][0] == lst2[n][0].conjugate():
                numberofreals += 1
                lst2[n] = (lst2[n][0].numerical_approx(30), 'real')
                if lst2[n][0] == 1:
                    numberofunits += 1
            else:
                lst2[n] = (lst2[n][0].abs().numerical_approx(30), 'cplx')
                if lst2[n][0] == 1:
                    numberofunits += 1
        print(i, numberofreals, numberofunits, is_palindrome(lst[i]), lst[i], lst2)

def t(a,b):
    '''helper function for f_red (see below)'''
    return b*(-1)**a/(math.factorial(a))*(math.factorial(b-a-1))/(math.factorial(b-2*a))

def f_red(f):
    '''Given a reciprocal polynomial f of degree 2n in the variable x, returns the polynomial f_R of degree n in the 
    variable y = x+x^{-1} such that f(x) = x^n f_R(y)'''
    if not is_palindrome(f):
        return "This is not a palindrome"
    else: 
        flist = f.list()[:(len(f.list())+1)/2]
        deg = len(flist)-1
        alist = flist[:]
        for i in range(deg+1):
            for l in range(1, int(math.floor((i)/2)+1)):
                alist[i] += flist[i-2*l]*t(l, deg-i+2*l)
        return S(list(reversed(alist)))

def symandskew(f):
    '''Given a polynomial, returns its symmetric and skewsymmetric part separately (in the variable y)'''
    flist = f.list()
    deg = len(flist)-1
    skew = [0]*(deg+1)
    sym = [0]*(deg+1)
    skew[1::2] = flist[1::2]
    sym[::2] = flist[::2]
    return(S(sym), S(skew))

For example, $\mathrm{F}_4$ (the Littlewood polynomials of degree 4), $\mathrm{Ir}_6$ (the irreducible Littlewood polynomials of degree 6), and $\mathrm{Sq}_6$ (the irreducible Littlewood polynomials of degree 6 with square discriminant) can be computed as follows:

In [2]:
F(4)

[x^4 + x^3 + x^2 + x + 1,
 x^4 + x^3 - x^2 + x + 1,
 x^4 + x^3 + x^2 - x + 1,
 x^4 + x^3 - x^2 - x + 1,
 x^4 + x^3 + x^2 + x - 1,
 x^4 + x^3 - x^2 + x - 1,
 x^4 + x^3 + x^2 - x - 1,
 x^4 + x^3 - x^2 - x - 1]

In [1]:
realparts = []
imagparts = []

for f in F(15):
    froots = f.roots(ring=CDF)
    for r in froots:
            realparts.append(r[0].real())
            imagparts.append(r[0].imag())

fig = plt.figure(figsize=(21, 13))
ax = fig.add_subplot(1,1,1)
plt.scatter(realparts, imagparts, s=0.05, color='red')
plt.show()

NameError: name 'F' is not defined

In [19]:
plt.close('all')

In [3]:
Ir(6)

[x^6 + x^5 + x^4 + x^3 + x^2 + x + 1,
 x^6 + x^5 + x^4 - x^3 + x^2 + x + 1,
 x^6 + x^5 - x^4 - x^3 + x^2 + x + 1,
 x^6 + x^5 - x^4 + x^3 - x^2 + x + 1,
 x^6 + x^5 + x^4 - x^3 - x^2 + x + 1,
 x^6 + x^5 - x^4 - x^3 - x^2 + x + 1,
 x^6 + x^5 + x^4 + x^3 + x^2 - x + 1,
 x^6 + x^5 + x^4 - x^3 + x^2 - x + 1,
 x^6 + x^5 - x^4 - x^3 + x^2 - x + 1,
 x^6 + x^5 + x^4 + x^3 - x^2 - x + 1,
 x^6 + x^5 - x^4 + x^3 - x^2 - x + 1,
 x^6 + x^5 - x^4 - x^3 - x^2 - x + 1,
 x^6 + x^5 + x^4 + x^3 + x^2 + x - 1,
 x^6 + x^5 - x^4 + x^3 + x^2 + x - 1,
 x^6 + x^5 + x^4 - x^3 + x^2 + x - 1,
 x^6 + x^5 - x^4 + x^3 - x^2 + x - 1,
 x^6 + x^5 + x^4 - x^3 - x^2 + x - 1,
 x^6 + x^5 - x^4 - x^3 - x^2 + x - 1,
 x^6 + x^5 - x^4 + x^3 + x^2 - x - 1,
 x^6 + x^5 + x^4 - x^3 + x^2 - x - 1,
 x^6 + x^5 - x^4 - x^3 + x^2 - x - 1,
 x^6 + x^5 + x^4 + x^3 - x^2 - x - 1,
 x^6 + x^5 - x^4 + x^3 - x^2 - x - 1,
 x^6 + x^5 + x^4 - x^3 - x^2 - x - 1]

In [17]:
Sq(6)

[x^6 + x^5 - x^4 + x^3 - x^2 + x + 1,
 x^6 + x^5 - x^4 - x^3 - x^2 + x + 1,
 x^6 + x^5 + x^4 - x^3 - x^2 + x - 1]

Some more advanced stuff: given a reciprocal polynomial $f(x)$ of degree $2n$, calculate the polynomial $f_{\text{red}}(y)$ in the variable $y = x+x^{-1}$ such that $f(x) = x^n f_{\text{red}}(y)$, and split the result in its symmetric and skewsymmetric part:

In [18]:
f_red(x^10+x^9-x^8-x^7-x^6-x^5-x^4-x^3-x^2+x+1)

y^5 + y^4 - 6*y^3 - 5*y^2 + 7*y + 3

In [19]:
symandskew(y^4 + y^3 - 5*y^2 - 4*y + 3)

(y^4 - 5*y^2 + 3, y^3 - 4*y)

Calculate the fraction $\Delta(f)/\Delta(f_{\text{red}})^2$ for all reciprocal polynomials of some degree, here the irreducible ones of degree 24 with square discriminant:

In [21]:
for i in range(5,20):
    print (y^8+i*y^6+i*y^4+i*y^2+1).discriminant().factor()

2^8 * 3^2 * 13^4 * 17^2
2^24 * 5^6
2^8 * 5^2 * 23^2 * 29^4
2^24 * 3^2 * 5^4 * 13^2
2^8 * 7^2 * 29^2 * 53^4
2^32 * 17^4
2^8 * 3^4 * 5^6 * 7^2 * 17^4
2^24 * 5^2 * 13^4 * 19^2
2^8 * 5^12 * 11^2 * 41^2
2^24 * 3^2 * 11^2 * 37^4
2^8 * 13^2 * 47^2 * 173^4
2^24 * 5^12 * 7^2
2^8 * 3^2 * 5^2 * 53^2 * 229^4
2^30 * 5^4 * 7^2 * 13^4
2^8 * 17^2 * 59^2 * 293^4


The function realandcplx does the following. Given a list of polynomials, returns for each polynomial in the list its index, number of real roots, numbermof unimodular roots, whether it is reciprocal or not, the polynomial itself, and a list of tuples of the roots (in the real case) or the absolute value of the root (in the complex case) with specification of them being either real or complex.

In [22]:
realandcplx(Sq(8))

(0, 0, 4, True, x^8 + x^7 + x^6 - x^5 + x^4 - x^3 + x^2 + x + 1, [(1.4868025, 'cplx'), (1.4868025, 'cplx'), (0.67258427, 'cplx'), (0.67258427, 'cplx'), (1.0000000, 'cplx'), (1.0000000, 'cplx'), (1.0000000, 'cplx'), (1.0000000, 'cplx')])
(1, 0, 4, True, x^8 + x^7 - x^6 - x^5 + x^4 - x^3 - x^2 + x + 1, [(1.3216632, 'cplx'), (1.3216632, 'cplx'), (0.75662244, 'cplx'), (0.75662244, 'cplx'), (1.0000000, 'cplx'), (1.0000000, 'cplx'), (1.0000000, 'cplx'), (1.0000000, 'cplx')])
(2, 4, 4, True, x^8 + x^7 - x^6 - x^5 - x^4 - x^3 - x^2 + x + 1, [(-1.4874760, 'real'), (-0.67227975, 'real'), (0.80511954, 'real'), (1.2420516, 'real'), (1.0000000, 'cplx'), (1.0000000, 'cplx'), (1.0000000, 'cplx'), (1.0000000, 'cplx')])
(3, 0, 0, False, x^8 + x^7 + x^6 + x^5 + x^4 - x^3 + x^2 - x + 1, [(1.3592358, 'cplx'), (1.3592358, 'cplx'), (0.91378670, 'cplx'), (0.91378670, 'cplx'), (1.0943473, 'cplx'), (1.0943473, 'cplx'), (0.73570753, 'cplx'), (0.73570753, 'cplx')])
(4, 0, 0, False, x^8 + x^7 + x^6 + x^5 - x^4 - 